#Introduction to Machine Learning: Final Project

Léo Dupire | ld2494

##Load Data

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# %cd /content/drive/MyDrive/NYU

In [ ]:
# !pip install torch
# !pip install torchvision
# !pip install opencv-python
# !pip install pickle-mixin
# !pip install pandas
# !pip install matplotlib
# !pip install sklearn
# !pip install -U scikit-learn scipy matplotlib
# !pip install statsmodels

In [ ]:
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor
import numpy as np
import cv2
import pickle as pkl
import random
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
class LazyLoadDataset(Dataset):
  def __init__(self, path, train=True, transform=None):
    self.transform = transform
    self.train = train
    path = path + ("train/" if train else "test/")

    self.pathX = path + "X/"
    self.pathY = path + "Y/"

    self.data = os.listdir(self.pathX)

  def __getitem__(self, idx):
    f = self.data[idx]

    img0 = cv2.imread(self.pathX + f + "/rgb/0.png")
    img1 = cv2.imread(self.pathX + f + "/rgb/1.png")
    img2 = cv2.imread(self.pathX + f + "/rgb/2.png")

    if self.transform is not None:
      img0 = self.transform(img0)
      img1 = self.transform(img1)
      img2 = self.transform(img2)

    depth = np.load(self.pathX + f + "/depth.npy")

    field_id = pkl.load(open(self.pathX + f + "/field_id.pkl", "rb"))

    Y = None
    if self.train:
      Y = np.load(self.pathY + f + ".npy")

    return (img0, img1, img2, depth, int(field_id)), Y
  
  def __len__(self):
    return len(self.data_folders)

In [3]:
# train = LazyLoadDataset("./lazydata/")
# test = LazyLoadDataset("./lazydata/", train=False)

In [3]:
# define custom transform function
transform = transforms.Compose([
    transforms.ToTensor()
])

###Download Data (Only Once)

In [ ]:
# X_img = []
# X_depth = []
# y = []

# for i in range(500): #len(train.data)):
#   print(i) 
#   (temp_img0, temp_img1, temp_img2, temp_depth, temp_field_id), temp_Y = train[i]

#   X_img.append(np.array(transform(temp_img0)))
#   # X_depth.append(temp_depth)
#   y.append(temp_Y)

In [ ]:
# torch.save(X_img, "X_img.pt")
# torch.save(y, "y.pt")

###Load Data (If Needed)

In [5]:
train_X_FULL = np.array(torch.load('./Desktop/Data_ML/train_X_FULL.pt'))
# train_X1_FULL = np.array(torch.load('./Desktop/Data_ML/train_X1_FULL.pt'))
# train_X2_FULL = np.array(torch.load('./Desktop/Data_ML/train_X2_FULL.pt'))
train_DEPTH_FULL = np.array(torch.load('./Desktop/Data_ML/train_DEPTH_FULL.pt'))
train_y_FULL = np.array(torch.load('./Desktop/Data_ML/train_y_FULL.pt'))

In [6]:
# These will go into the model(s)
train_img = torch.tensor(np.array(train_X_FULL).reshape([len(train_X_FULL), 3, 224, 224]))
# train_img1 = torch.tensor(np.array(train_X1_FULL).reshape([len(train_X_FULL), 3, 224, 224]))
# train_img2 = torch.tensor(np.array(train_X2_FULL).reshape([len(train_X_FULL), 3, 224, 224]))
train_depth = torch.tensor(np.array(train_DEPTH_FULL).reshape([len(train_X_FULL), 3, 224, 224]))
train_Y = torch.tensor(np.array(train_y_FULL))

In [38]:
train_img.shape

torch.Size([3396, 3, 224, 224])

##Model

In [19]:
# Creating a CNN class
class ConvNeuralNet(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(ConvNeuralNet, self).__init__()
        
        self.conv_layer1 = nn.Conv2d(in_channels=3, out_channels=32, kernel_size=3)

        self.conv_layer2 = nn.Conv2d(in_channels=32, out_channels=32, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.bn1 = nn.BatchNorm2d(32)
        
        self.conv_layer3 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.fc1 = nn.Linear(179776, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 12)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)

        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        out = self.bn1(out.reshape(1, 32, 110, 110))
        
        out = self.conv_layer3(out)
        
        out = self.conv_layer4(out)
        out = self.max_pool2(out)

        # Flatten manually
        out = out.reshape(out.size(0), -1)
        out = out.reshape(-1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)

        return out

In [20]:
# Define relevant variables for the ML task
batch_size = 64
num_classes = 12
learning_rate = 0.001
num_epochs = 10
train_size = len(train_img)

# Model for top RGB Image
model_RGBTop = ConvNeuralNet(num_classes)

# Model for Depth Image
model_DEPTH = ConvNeuralNet(num_classes)

# Set Loss function with criterion + criterion
criterion = nn.MSELoss()
optimizer_RGBTop = torch.optim.SGD(model_RGBTop.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  
optimizer_DEPTH = torch.optim.SGD(model_DEPTH.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

###Train RGB Top Image Model

In [21]:
# Randomize order of images
order = list(range(len(train_img)))
random.shuffle(order)

# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i in range(train_size):
        if (i%10 == 0):
          print(i, end=" ")

        # Move tensors to the configured device
        images = train_img[order[i]]
        labels = train_Y[order[i]]

        # Forward pass
        outputs = model_RGBTop(images.float())
        loss = torch.sqrt(criterion(outputs, labels.float()))
        
        # Backward and optimize
        optimizer_RGBTop.zero_grad()
        loss.backward()
        optimizer_RGBTop.step()

    print('\nEpoch [{}/{}], Loss: {:.6f}'.format(epoch+1, num_epochs, loss.item()))
    print()

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2

###Train Depth Model

In [83]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i in range(train_size):
        if (i%10 == 0):
          print(i, end=" ")

        # Move tensors to the configured device
        images = train_depth[order[i]]
        labels = train_Y[order[i]]

        # Forward pass
        outputs = model_DEPTH(images.float())
        loss = torch.sqrt(criterion(outputs, labels.float()))
        
        # Backward and optimize
        optimizer_DEPTH.zero_grad()
        loss.backward()
        optimizer_DEPTH.step()

    print('\nEpoch [{}/{}], Loss: {:.6f}'.format(epoch+1, num_epochs, loss.item()))
    print()

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 850 860 870 880 890 900 910 920 930 940 950 960 970 980 990 1000 1010 1020 1030 1040 1050 1060 1070 1080 1090 1100 1110 1120 1130 1140 1150 1160 1170 1180 1190 1200 1210 1220 1230 1240 1250 1260 1270 1280 1290 1300 1310 1320 1330 1340 1350 1360 1370 1380 1390 1400 1410 1420 1430 1440 1450 1460 1470 1480 1490 1500 1510 1520 1530 1540 1550 1560 1570 1580 1590 1600 1610 1620 1630 1640 1650 1660 1670 1680 1690 1700 1710 1720 1730 1740 1750 1760 1770 1780 1790 1800 1810 1820 1830 1840 1850 1860 1870 1880 1890 1900 1910 1920 1930 1940 1950 1960 1970 1980 1990 2000 2010 2020 2030 2040 2050 2060 2070 2080 2090 2100 2110 2120 2130 2140 2150 2160 2170 2180 2190 2200 2210 2

###RGBD Model

In [7]:
# Merge two depth 3 images into one of depth 6
train_merge = torch.cat((train_img, train_depth), 1)

In [8]:
train_merge.shape

torch.Size([3396, 6, 224, 224])

In [13]:
# Creating a CNN class
class RGBD(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(RGBD, self).__init__()
        
        self.conv_layer1 = nn.Conv2d(in_channels=6, out_channels=64, kernel_size=4)

        self.conv_layer2 = nn.Conv2d(in_channels=64, out_channels=64, kernel_size=3)
        self.max_pool1 = nn.MaxPool2d(kernel_size = 2, stride = 2)
        self.bn1 = nn.BatchNorm2d(64)
        
        self.conv_layer3 = nn.Conv2d(in_channels=64, out_channels=128, kernel_size=3)
        self.conv_layer4 = nn.Conv2d(in_channels=128, out_channels=128, kernel_size=3)
        self.max_pool2 = nn.MaxPool2d(kernel_size = 2, stride = 2)

        self.fc1 = nn.Linear(346112, 512)
        self.relu1 = nn.ReLU()
        self.fc2 = nn.Linear(512, 128)
        self.relu2 = nn.ReLU()
        self.fc3 = nn.Linear(128, 12)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)

        out = self.conv_layer2(out)
        out = self.max_pool1(out)
        out = self.bn1(out.reshape(1, 64, 109, 109))
        
        out = self.conv_layer3(out)
        
        out = self.conv_layer4(out)
        out = self.max_pool2(out)

        # Flatten manually
        out = out.reshape(out.size(0), -1)
        out = out.reshape(-1)
        
        out = self.fc1(out)
        out = self.relu1(out)
        out = self.fc2(out)
        out = self.relu2(out)
        out = self.fc3(out)

        return out

In [14]:
# Define relevant variables for the ML task
batch_size = 64
num_classes = 12
learning_rate = 0.001
num_epochs = 10
train_size = 500 #len(train_img)

# Model for top RGB Image
model_RGBD = RGBD(num_classes)

# Set Loss function with criterion + criterion
criterion = nn.MSELoss()
optimizer_RGBD = torch.optim.SGD(model_RGBD.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9)  

In [16]:
# Randomize order of images
order = list(range(len(train_img)))
random.shuffle(order)

# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i in range(train_size):
        if (i%10 == 0):
          print(i, end=" ")

        # Move tensors to the configured device
        images = train_merge[order[i]]
        labels = train_Y[order[i]]

        # Forward pass
        outputs = model_RGBD(images.float())
        loss = torch.sqrt(criterion(outputs, labels.float()))
        
        # Backward and optimize
        optimizer_RGBD.zero_grad()
        loss.backward()
        optimizer_RGBD.step()

    print('\nEpoch [{}/{}], Loss: {:.6f}'.format(epoch+1, num_epochs, loss.item()))
    print()

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 
Epoch [1/5], Loss: 0.008423

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 
Epoch [2/5], Loss: 0.007043

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 
Epoch [3/5], Loss: 0.006709

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 
Epoch [4/5], Loss: 0.006309

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330

###Train Merger Model

####First get the predictions from the RGBTop and Depth models for the full train dataset

In [ ]:
RGBTop_pred = []
DEPTH_pred = []
with torch.no_grad():
    for i in range(len(train_img)):
        if (i%10 == 0):
          print(i, end=" ")
        IMG = train_img[i]
        DPT = train_depth[i]

        outputs_IMG = model_RGBTop(IMG)
        outputs_DPT = model_DEPTH(DPT)

        RGBTop_pred.append(outputs_IMG)
        DEPTH_pred.append(outputs_DPT)

In [ ]:
RGBTop = np.array(pd.DataFrame.from_records(np.array(RGBTop_pred))).astype(float)
DEPTH = np.array(pd.DataFrame.from_records(np.array(DEPTH_pred))).astype(float)

In [ ]:
merge_data = []

for i in range(len(train_img)):
  temp = []
  for j in range(12):
    temp.append([RGBTop[i][j], DEPTH[i][j]])
  merge_data.append(np.array(temp).T)

merge_data = torch.tensor(merge_data).reshape([3396, 1, 2, 12])

In [ ]:
merge_data.shape

torch.Size([3396, 1, 2, 12])

In [95]:
class MergerCNN(nn.Module):
	#  Determine what layers and their order in CNN object 
    def __init__(self, num_classes):
        super(MergerCNN, self).__init__()
        
        # Add more neurons to prevent 'unlearning'
        self.conv_layer1 = nn.Conv2d(in_channels=1, out_channels=100, kernel_size=3, padding=1) 
        self.batch_norm1 = nn.BatchNorm2d(100)
        
        self.conv_layer2 = nn.Conv2d(in_channels=100, out_channels=200, kernel_size=1)
        self.batch_norm2 = nn.BatchNorm2d(200)
        self.relu1 = nn.LeakyReLU(0.1)
        
        self.conv_layer3 = nn.Conv2d(in_channels=200, out_channels=200, kernel_size=1)
        self.batch_norm3 = nn.BatchNorm2d(200)
        # self.dropout2 = nn.Dropout(p=0.1)
        self.relu2 = nn.LeakyReLU(0.1)

        self.fc1 = nn.Linear(12, 64)
        self.relu3 = nn.LeakyReLU(0.1)
        self.fc2 = nn.Linear(64, 12)
    
    # Progresses data across layers    
    def forward(self, x):
        out = self.conv_layer1(x)
        out = self.batch_norm1(out.reshape([1, 100, 2, 12]))

        out = self.conv_layer2(out)
        out = self.batch_norm2(out)
        out = self.relu1(out)
        
        out = self.conv_layer3(out)
        out = self.batch_norm3(out)
        # out = self.dropout2(out)
        out = self.relu2(out)
 
        out = self.fc1(out)
        out = self.relu3(out)
        out = self.fc2(out)

        return out

In [96]:
# Define relevant variables for the ML task
num_classes = 12
learning_rate = 0.001
num_epochs = 30
train_size = len(train_img)

# Model for top RGB Image
model_MERGE = MergerCNN(num_classes)

# Set Loss function with criterion + criterion
criterion = nn.MSELoss()
optimizer_MERGE = torch.optim.SGD(model_MERGE.parameters(), lr=learning_rate, weight_decay = 0.005, momentum = 0.9) 

In [97]:
# We use the pre-defined number of epochs to determine how many iterations to train the network on
for epoch in range(num_epochs):
	#Load in the data in batches using the train_loader object
    for i in range(train_size):
        if (i%100 == 0):
          print(i, end=" ")

        # Move tensors to the configured device
        images = merge_data[order[i]]
        labels = train_Y[order[i]]

        # Forward pass
        outputs = model_MERGE(images.float())
        loss = torch.sqrt(criterion(outputs, labels.float()))
        
        # Backward and optimize
        optimizer_MERGE.zero_grad()
        loss.backward()
        optimizer_MERGE.step()

    print('\nEpoch [{}/{}], Loss: {:.6f}'.format(epoch+1, num_epochs, loss.item()))
    print()

0 

/Users/leodupire/miniforge3/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([12])) that is different to the input size (torch.Size([1, 512, 2, 12])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 
Epoch [1/30], Loss: 0.025206

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 
Epoch [2/30], Loss: 0.024033

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 
Epoch [3/30], Loss: 0.023953

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 
Epoch [4/30], Loss: 0.024014

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300 1400 1500 1600 1700 1800 1900 2000 2100 2200 2300 2400 2500 2600 2700 2800 2900 3000 3100 3200 3300 
Epoch [5/30], Loss: 0.024016

0 100 200 300 400 500 600 700 800 900 1000 1100 1200 1300

##Evaluation

###Download Data (Only Once)

In [ ]:
# test_img = []
# test_depth = []
# test_ID = []

# for i in range(len(test.data)):
#   print(i)
#   (temp_img0, temp_img1, temp_img2, temp_depth, temp_field_id), temp_Y = test[i]

#   test_img.append(np.array(transform(temp_img0)))
#   # X_depth.append(temp_depth)
#   test_ID.append(temp_field_id)

In [ ]:
# torch.save(test_img, "test_img.pt")

###Load Data (If Needed)

In [66]:
test_imgs = torch.tensor(np.array(torch.load('./Desktop/Data_ML/test_IMG.pt')))
test_depth = torch.tensor(np.array(torch.load('./Desktop/Data_ML/test_DEPTH.pt')))
test_ids = torch.tensor(np.array(torch.load('./Desktop/Data_ML/test_ID.pt')))

In [67]:
test_merge = torch.cat((test_imgs, test_depth.reshape([849, 3, 224, 224])), 1)

In [68]:
# RGBDepth Predictions
predictions_RGBD = []
with torch.no_grad():
    for i in range(len(test_merge)):
        if (i%10 == 0):
          print(i, end=" ")
        images = test_merge[i]
        outputs = model_RGBD(images)
        predictions_RGBD.append(outputs)

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 

In [59]:
# RGBTop and Depth predictions
predictions = []
with torch.no_grad():
    for i in range(len(test_imgs)):
        if (i%10 == 0):
          print(i, end=" ")
        images = test_imgs[i]
        outputs = model_RGBTop(images)
        predictions.append(outputs)

0 10 20 30 40 50 60 70 80 90 100 110 120 130 140 150 160 170 180 190 200 210 220 230 240 250 260 270 280 290 300 310 320 330 340 350 360 370 380 390 400 410 420 430 440 450 460 470 480 490 500 510 520 530 540 550 560 570 580 590 600 610 620 630 640 650 660 670 680 690 700 710 720 730 740 750 760 770 780 790 800 810 820 830 840 

In [69]:
import pickle
import pandas as pd

outfile = './Desktop/Data_ML/submission.csv'

output_file = open(outfile, 'w')

titles = ['ID', 'FINGER_POS_1', 'FINGER_POS_2', 'FINGER_POS_3', 'FINGER_POS_4', 'FINGER_POS_5', 'FINGER_POS_6',
         'FINGER_POS_7', 'FINGER_POS_8', 'FINGER_POS_9', 'FINGER_POS_10', 'FINGER_POS_11', 'FINGER_POS_12']

df = pd.concat([pd.DataFrame(np.array(test_ids)), pd.DataFrame.from_records(np.array(predictions_RGBD)).astype(float)], axis = 1, names = titles)
df.columns = titles
df.to_csv(outfile, index = False)
print("Written to csv file {}".format(outfile))

Written to csv file ./Desktop/Data_ML/submission.csv


/var/folders/01/4rw9wbtn40s22slt7887wq7r0000gn/T/ipykernel_32605/2504348550.py:11: FutureWarning: The input object of type 'Tensor' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Tensor', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  df = pd.concat([pd.DataFrame(np.array(test_ids)), pd.DataFrame.from_records(np.array(predictions_RGBD)).astype(float)], axis = 1, names = titles)
/var/folders/01/4rw9wbtn40s22slt7887wq7r0000gn/T/ipykernel_32605/2504348550.py:11: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=o